# **Closing and small particle removal**
---
<font size =4>This notebook has been created to perform the post-processing of the previously segmented cysts in the notebook *1_U-Net_2D_Multilabel_ZeroCostDL4Mic.ipynb*.

<font size =4>The second section is about connecting the notebook to the user's Google Drive, in case of using Google Colab.

<font size =4>In the third section masks are processed in order to achive a better rersult. Specifically, closing of holes and subsequence particle removal is performed.

<font size =4> Finally, in the forth section, the masks are saved as image sequence.

---

<font size =4>*Author*: Itsaso Vitoria

<font size =4>*Date*: *2023, September 8*

# **1. Load key dependencies**
-----

In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB 991.0 kB/s eta 0:00:39
   - -------------------------------------- 1.4/38.1 MB 14.4 MB/s eta 0:00:03
   --- ------------------------------------ 2.9/38.1 MB 20.6 MB/s eta 0:00:02
   ---- ----------------------------------- 4.5/38.1 MB 23.8 MB/s eta 0:00:02
   ----- ---------------------------------- 5.6/38.1 MB 23.7 MB/s eta 0:00:02
   ------- -------------------------------- 7.2/38.1 MB 25.5 MB/s eta 0:00:02
   --------- ------------------------------ 8.8/38.1 MB 28.1 MB/s eta 0:00:02
   ---------- ----------------------------- 10.3/38.1 MB 31.2 MB/s eta 0:00:01
   ------------ --------------------------- 11.8/38.1 MB 32.7 MB/s eta 0:00:01
   -

In [3]:
#@markdown ###Run to import necessary modules and functions.

# Import necessary modules
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from PIL import Image

# Function for computing time in minutes
def format_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{int(minutes)} minutes and {int(remaining_seconds)} seconds"

# **2. Initialise the Colab session** <font size = 4>(in case you use Google Drive)

---

## **2.1. Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelerator: GPU** *(Graphics processing unit)*

In [4]:
#@markdown ##Run this cell to check if you have GPU access
import tensorflow as tf
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.')
  print('Did you change your runtime ?')
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')

else:
  print('You have GPU access')
  !nvidia-smi

You have GPU access
Tue Oct 10 11:01:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.23                 Driver Version: 536.23       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:2D:00.0  On |                  N/A |
|  0%   45C    P2              68W / 350W |   1667MiB / 24576MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-----

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **3. Closing and holes removal**
----
<font size = 4>

In [8]:
start_process = time.time()

#@markdown ### Set the folder path where the images are located.
imPath = 'E:/AAV para enfermedades renales/LSFM combined images/CystSegmentation/InitialSegMasks/MacroSPIM2/'+\
            'InitialSeg_KidneyR2Right_LectinRed_RLSamples2021' #@param {type:"string"}

#@markdown ### Define the save pathl
savePath = 'E:/AAV para enfermedades renales/LSFM combined images/CystSegmentation/FinalSegMasks/MacroSPIM2/'+\
            'FinalCystSeg_KidneyR2Right_LectinRed_RLSamples2021'#@param {type:"string"}

# Ensure the save directory exists
os.makedirs(savePath, exist_ok=True)

# Get a list of all the files in the folder
file_list = os.listdir(imPath)

# Process each image in the folder
for file_name in file_list:
    # Construct the full file path
    file_path = os.path.join(imPath, file_name)

    # Load the binary mask image
    binary_mask = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    # Perform your desired image processing operations here
    kernel = np.ones((6, 6), np.uint8)
    closed_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

    kernel = np.ones((7, 7), np.uint8)
    binary_mask = cv2.erode(closed_mask, kernel, iterations=2)
    binary_mask = cv2.dilate(binary_mask, kernel, iterations=2)

    # Save the processed binary mask as a .tif file with the same name
    maskPath = os.path.join(savePath, file_name)
    slide = Image.fromarray(binary_mask)
    slide.save(maskPath)

end_process = time.time()
print("Time for postprocessing all the masks:", format_minutes_seconds(end_process - start_process))

Time for postprocessing all the masks: 3 minutes and 17 seconds
